In [1]:
import nltk

from utils.dataset import DataSet
from feature_engineering import refuting_features, polarity_features, hand_features, gen_or_load_feats, word_overlap_features, get_tokenized_lemmas
training_dataset = DataSet("train")
testing_dataset = DataSet("competition_test")
from nltk.tokenize import word_tokenize
# competition_dataset

LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
LABELS_RELATED = ['unrelated','related']

Reading dataset
Total stances: 49972
Total bodies: 1683
Reading dataset
Total stances: 25413
Total bodies: 904


In [2]:
h, b, y = [],[],[]
name = "comp"
x = 0
training_df = training_dataset.to_dataframe()
training_df.head()

testing_df = testing_dataset.to_dataframe()

# X_overlap = gen_or_load_feats(word_overlap_features, h, b, "features/overlap."+name+".npy")
# X_refuting = gen_or_load_feats(refuting_features, h, b, "features/refuting."+name+".npy")
# X_polarity = gen_or_load_feats(polarity_features, h, b, "features/polarity."+name+".npy")
# X_hand = gen_or_load_feats(hand_features, h, b, "features/hand."+name+".npy")

# X = np.c_[X_hand, X_polarity, X_refuting, X_overlap]
# X

In [3]:
training_df.head()


# Tokenizing and lemmatizing everything
training_df['processed_headlines'] = training_df.apply(lambda row: get_tokenized_lemmas(row['Headlines']), axis=1)
training_df['processed_bodies'] = training_df.apply(lambda row: get_tokenized_lemmas(row['Bodies']), axis=1)

testing_df['processed_headlines'] = testing_df.apply(lambda row: get_tokenized_lemmas(row['Headlines']), axis=1)
testing_df['processed_bodies'] = testing_df.apply(lambda row: get_tokenized_lemmas(row['Bodies']), axis=1)





In [4]:
testing_df.head()

,Stances,Headlines,Bodies,processed_headlines,processed_bodies
0,3,ferguson riots pregnant woman loses eye after ...,a respected senior french police officer inves...,ferguson riot pregnant woman loses eye after c...,a respected senior french police officer inves...
1,3,crazy conservatives are sure a gitmo detainee ...,dave morin s social networking company path is...,crazy conservative are sure a gitmo detainee k...,dave morin s social networking company path is...
2,3,a russian guy says his justin bieber ringtone ...,a bereaved afghan mother took revenge on the t...,a russian guy say his justin bieber ringtone s...,a bereaved afghan mother took revenge on the t...
3,3,zombie cat buried kitty believed dead meows ba...,hewlett packard is officially splitting in two...,zombie cat buried kitty believed dead meow bac...,hewlett packard is officially splitting in two...
4,3,argentina s president adopts boy to end werewo...,an airline passenger headed to dallas was remo...,argentina s president adopts boy to end werewo...,an airline passenger headed to dallas wa remov...


In [5]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer=TfidfVectorizer(use_idf=True)
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Getting the TFs for Training data
cv = CountVectorizer(max_features=5000)
headlines_count = cv.fit_transform(training_df['processed_headlines'])
bodies_count = cv.transform(training_df['processed_bodies'])

headlines_tf = TfidfTransformer(use_idf=False).fit_transform(headlines_count)
bodies_tf = TfidfTransformer(use_idf=False).transform(bodies_count)

# Getting the TFs for Testing Data
headlines_count_test = cv.transform(testing_df['processed_headlines'])
bodies_count_test = cv.transform(testing_df['processed_bodies'])

tfidf_transformer = TfidfTransformer(use_idf=False)
headlines_tf = tfidf_transformer.fit_transform(headlines_count)
bodies_tf = tfidf_transformer.transform(bodies_count)

headlines_tf_test  = tfidf_transformer.transform(headlines_count_test)
bodies_tf_test = tfidf_transformer.transform(bodies_count_test)


headline_vectors = tfidf_vectorizer.fit_transform(training_df['processed_headlines']).todense()
body_vectors = tfidf_vectorizer.transform(training_df['processed_bodies'])

headline_vectors_test = tfidf_vectorizer.fit_transform(testing_df['processed_headlines']).todense()
body_vectors_test = tfidf_vectorizer.transform(testing_df['processed_bodies'])


print(type(headline_vectors))
print(headline_vectors.shape)
print(body_vectors_test.shape)



<class 'numpy.matrix'>
(49972, 3093)
(25413, 2116)


In [6]:
arr = headlines_count[0].toarray()[0]
headlines_count

<49972x3093 sparse matrix of type '<class 'numpy.int64'>'
	with 538152 stored elements in Compressed Sparse Row format>

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
cosine_similarity_list = []
cosine_similarity_list_test = []

full_feature_train = []
full_feature_test = []
for i in range(headline_vectors.shape[0]):
    cosine_similarity_temp = cosine_similarity(headline_vectors[i], body_vectors[i])
    cosine_similarity_list.append(cosine_similarity_temp)
    headline_tf_vector = headlines_count[i].toarray()[0]
    body_tf_vector = bodies_count[i].toarray()[0]
    full_feature = np.hstack(( cosine_similarity_temp[0],headline_tf_vector,body_tf_vector )).ravel()
    full_feature_train.append(full_feature)
    
for i in range(headline_vectors_test.shape[0]):
    cosine_similarity_temp = cosine_similarity(headline_vectors_test[i], body_vectors_test[i])
    cosine_similarity_list_test.append(cosine_similarity_temp)
    headline_tf_vector = headlines_count[i].toarray()[0]
    body_tf_vector = bodies_count[i].toarray()[0]
    full_feature = np.hstack(( cosine_similarity_temp[0],headline_tf_vector,body_tf_vector )).ravel()
    full_feature_test.append(full_feature)




# cs_l = []
# for element in cosine_similarity_list:
#     cs_l.append(element[0][0])


# cs_l = np.array(cs_l)

In [8]:
full_feature_test[1]

array([0., 0., 0., ..., 0., 0., 0.])

In [9]:
cs_l_test = []
# for element in consine_similarity_list_test:
#     cs_l_test.append(element[0][0])
    
# cs_l = np.array(cs_l)
# cs_l = np.array(cs_l)
# cs_l.shape

cs_l_test = np.array(cs_l_test)

full_feature = np.array(full_feature_train)
full_feature_test_1 = np.array(full_feature_test)
full_feature.shape

(49972, 6187)

In [10]:
print(cs_l[0])

NameError: name 'cs_l' is not defined

In [ ]:
from keras.utils import np_utils
y_train_categorical = np_utils.to_categorical(training_df['Stances'])
y_test_categorical = np_utils.to_categorical(testing_df['Stances'])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers.core import Dense, Dropout, Activation

model = Sequential()

model.add(Dense(500, input_shape=(6187,)))

model.add(Activation('relu'))

model.add(Dropout(0.5))

# model.add(Dense(500))

# model.add(Activation('relu'))

# model.add(Dropout(0.5))

model.add(Dense(50))

model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(4))

model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
# cs_l.shape
model.fit(full_feature, y_train_categorical, batch_size=128, epochs=5, verbose=1)

In [ ]:
len(full_feature_test)

In [ ]:
results = model.evaluate(full_feature_test_1, y_test_categorical, batch_size=1)
# predictions = model.predict(full_feature_test)


In [ ]:
from utils.score import report_score, generate_answer_csv
predicted = []
print(predictions.shape)
for prediction in predictions:
    max_val = np.argmax(prediction)
    predicted.append(max_val)
predicted_vals = [LABELS[int(a)] for a in predicted]
actual = [LABELS[int(a)] for a in testing_df['Stances']]



report_score(actual, predicted_vals)